<a href="https://colab.research.google.com/github/cocoy02/BT4012_AML-Fraud-Detection/blob/main/GCN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from __future__ import print_function, division
import numpy as np
import pandas as pd

import os

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_recall_fscore_support, f1_score, classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import warnings
warnings.filterwarnings("ignore")
torch.manual_seed(15)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Import data

In [ ]:
df_classes = pd.read_csv("/content/drive/My Drive/BT4012 Team 8/Data/elliptic_txs_classes.csv")
df_edges = pd.read_csv("/content/drive/My Drive/BT4012 Team 8/Data/elliptic_txs_edgelist.csv")
df_features = pd.read_csv("/content/drive/My Drive/BT4012 Team 8/Data/elliptic_txs_features.csv", header=None)
df_features.columns = ['txId', 'time_step'] + [f'trans_feat_{i}' for i in range(93)] + [f'agg_feat_{i}' for i in range(72)]
#df_features = pd.merge(df_features,df_classes,left_on="txId",right_on="txId",how='left')
df_classes['class'] = df_classes['class'].apply(lambda x: '0' if x == "unknown" else x)

# GCN

In [ ]:
def load_data(classes,edgelist,features,start_ts,end_ts):
  num_features = features.shape[1]
  num_tx = features.shape[0]
  total_tx = list(classes.index)

  # select only the transactions which are labelled
  labelled_classes = classes[classes['class'] != '0']
  labelled_tx = list(labelled_classes.index)

  # to calculate a list of adjacency matrices for the different timesteps

  adj_mats = []
  features_labelled_ts = []
  classes_ts = []
  num_ts = 49 # number of timestamps from the paper

  for ts in range(start_ts, end_ts):
      features_ts = features[features['time_step'] == ts+1]
      tx_ts = list(features_ts.index)

      labelled_tx_ts = [tx for tx in tx_ts if tx in set(labelled_tx)]

      # adjacency matrix for all the transactions
      # we will only fill in the transactions of this timestep which have labels and can be used for training
      adj_mat = pd.DataFrame(np.zeros((num_tx, num_tx)), index = total_tx, columns = total_tx)

      edgelist_labelled_ts = edgelist.loc[edgelist.index.intersection(labelled_tx_ts).unique()]
      for i in range(edgelist_labelled_ts.shape[0]):
          adj_mat.loc[edgelist_labelled_ts.index[i], edgelist_labelled_ts.iloc[i]['txId2']] = 1

      adj_mat_ts = adj_mat.loc[labelled_tx_ts, labelled_tx_ts]
      features_l_ts = features.loc[labelled_tx_ts]

      adj_mats.append(adj_mat_ts)
      features_labelled_ts.append(features_l_ts)
      classes_ts.append(classes.loc[labelled_tx_ts])

  return adj_mats, features_labelled_ts, classes_ts

In [ ]:
adj_mats, features_labelled_ts, classes_ts = load_data(df_classes,df_edges,df_features,0,34)


# GCN

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import numpy as np
import time
import sys

class GraphConv(nn.Module):
    def __init__(self, in_features, out_features, activation  = 'relu', skip = False, skip_in_features = None):
        super(GraphConv, self).__init__()
        self.W = torch.nn.Parameter(torch.DoubleTensor(in_features, out_features))
        nn.init.xavier_uniform_(self.W)

        self.set_act = False
        if activation == 'relu':
            self.activation = nn.ReLU()
            self.set_act = True
        elif activation == 'softmax':
            self.activation = nn.Softmax(dim = 1)
            self.set_act = True
        else:
            self.set_act = False
            raise ValueError("activations supported are 'relu' and 'softmax'")

        self.skip = skip
        if self.skip:
            if skip_in_features == None:
                raise ValueError("pass input feature size of the skip connection")
            self.W_skip = torch.nn.Parameter(torch.DoubleTensor(skip_in_features, out_features))
            nn.init.xavier_uniform_(self.W)

    def forward(self, A, H_in, H_skip_in = None):
        # A must be an n x n matrix as it is an adjacency matrix
        # H is the input of the node embeddings, shape will n x in_features
        self.A = A
        self.H_in = H_in
        A_ = torch.add(self.A, torch.eye(self.A.shape[0]).double())
        D_ = torch.diag(A_.sum(1))
        # since D_ is a diagonal matrix,
        # its root will be the roots of the diagonal elements on the principle diagonal
        # since A is an adjacency matrix, we are only dealing with positive values
        # all roots will be real
        D_root_inv = torch.inverse(torch.sqrt(D_))
        A_norm = torch.mm(torch.mm(D_root_inv, A_), D_root_inv)
        # shape of A_norm will be n x n

        H_out = torch.mm(torch.mm(A_norm, H_in), self.W)
        # shape of H_out will be n x out_features

        if self.skip:
            H_skip_out = torch.mm(H_skip_in, self.W_skip)
            H_out = torch.add(H_out, H_skip_out)

        if self.set_act:
            H_out = self.activation(H_out)

        return H_out

# GCN 2 layers

In [ ]:
class GCN_2layer(nn.Module):
    def __init__(self, in_features, hidden_features, out_features, skip = False):
        super(GCN_2layer, self).__init__()
        self.skip = skip

        self.gcl1 = GraphConv(in_features, hidden_features)

        if self.skip:
            self.gcl_skip = GraphConv(hidden_features, out_features, activation = 'softmax', skip = self.skip,
                                  skip_in_features = in_features)
        else:
            self.gcl2 = GraphConv(hidden_features, out_features, activation = 'softmax')

    def forward(self, A, X):
        out = self.gcl1(A, X)
        if self.skip:
            out = self.gcl_skip(A, out, X)
        else:
            out = self.gcl2(A, out)

        return out

In [ ]:
num_features = 167
num_classes = 2
num_ts = 49
epochs = 15
lr = 0.001
max_train_ts = 34
train_ts = np.arange(max_train_ts)

#adj_mats, features_labelled_ts, classes_ts = dataSet

# 0 - illicit, 1 - licit
labels_ts = []
for c in classes_ts:
    labels_ts.append(np.array(c['class'] == '2', dtype = np.long))

gcn = GCN_2layer(num_features, 100, num_classes)
train_loss = nn.CrossEntropyLoss(weight = torch.DoubleTensor([0.7, 0.3]))
optimizer = torch.optim.Adam(gcn.parameters(), lr = lr)

# Training

for ts in train_ts:
    A = torch.tensor(adj_mats[ts].values)
    X = torch.tensor(features_labelled_ts[ts].values)
    L = torch.tensor(labels_ts[ts], dtype = torch.long)
    for ep in range(epochs):
        t_start = time.time()

        gcn.train()
        optimizer.zero_grad()
        out = gcn(A, X)

        loss = train_loss(out, L)
        train_pred = out.max(1)[1].type_as(L)
        acc = (train_pred.eq(L).double().sum())/L.shape[0]

        loss.backward()
        optimizer.step()

        sys.stdout.write("\r Epoch %d/%d Timestamp %d/%d training loss: %f training accuracy: %f Time: %s"
                         %(ep, epochs, ts, max_train_ts, loss, acc, time.time() - t_start)
                        )

torch.save(gcn.state_dict(), "/content/drive/My Drive/BT4012 Team 8/modelDir"+ "gcn_weights.pth")

 Epoch 14/15 Timestamp 33/34 training loss: 0.466245 training accuracy: 0.928155 Time: 0.11188364028930664

In [ ]:
from sklearn.metrics import f1_score, precision_score, recall_score
test_ts = np.arange(14)
adj_mats, features_labelled_ts, classes_ts = load_data(df_classes,df_edges,df_features, 35, 49)

# 0 - illicit, 1 - licit
labels_ts = []
for c in classes_ts:
    labels_ts.append(np.array(c['class'] == '2', dtype = np.long))

gcn = GCN_2layer(num_features, 100, num_classes)
gcn.load_state_dict(torch.load("/content/drive/My Drive/BT4012 Team 8/modelDir"+ "gcn_weights.pth"))

# Testing
test_accs = []
test_precisions = []
test_recalls = []
test_f1s = []

for ts in test_ts:
    A = torch.tensor(adj_mats[ts].values)
    X = torch.tensor(features_labelled_ts[ts].values)
    L = torch.tensor(labels_ts[ts], dtype = torch.long)

    gcn.eval()
    test_out = gcn(A, X)

    test_pred = test_out.max(1)[1].type_as(L)
    t_acc = (test_pred.eq(L).double().sum())/L.shape[0]
    test_accs.append(t_acc.item())
    test_precisions.append(precision_score(L, test_pred))
    test_recalls.append(recall_score(L, test_pred))
    test_f1s.append(f1_score(L, test_pred))

acc = np.array(test_accs).mean()
prec = np.array(test_precisions).mean()
rec = np.array(test_recalls).mean()
f1 = np.array(test_f1s).mean()

print("GCN - averaged accuracy: {}, precision: {}, recall: {}, f1: {}".format(acc, prec, rec, f1))

GCN - averaged accuracy: 0.9371110767475882, precision: 0.9371110767475882, recall: 1.0, f1: 0.9669427201086112
